In [1]:
from valkey_trader import TradePublisherDemo

publisher = TradePublisherDemo()
publisher.run()


📤 Connected to Valkey at localhost:6379. Publishing on 'trades'...
🆕 Published batch from Asteroid: {"timestamp": 1750929097, "trader": "Asteroid", "actions": [{"crypto": "ADA", "action": "sell", "quantity": 3.21}]}
🆕 Published batch from Exoplanet: {"timestamp": 1750929099, "trader": "Exoplanet", "actions": [{"crypto": "XRP", "action": "buy", "quantity": 0.13}, {"crypto": "ADA", "action": "buy", "quantity": 0.84}]}
🆕 Published batch from Asteroid: {"timestamp": 1750929120, "trader": "Asteroid", "actions": [{"crypto": "ADA", "action": "sell", "quantity": 0.05}]}
🆕 Published batch from Exoplanet: {"timestamp": 1750929120, "trader": "Exoplanet", "actions": [{"crypto": "ADA", "action": "buy", "quantity": 8.4}, {"crypto": "ETH", "action": "sell", "quantity": 7.9}]}

❌ Publisher stopped gracefully.
